In [9]:
#load the dataset
import pandas as pd
dataset = pd.read_csv('/content/hate_speech.csv')
dataset.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [10]:
dataset.shape

(5242, 3)

In [11]:
dataset.label.value_counts()

,count
label,
0,3000
1,2242


In [12]:
for index, tweet in enumerate(dataset["tweet"][10:15]):
  print(index+1,"-",tweet)

1 -  â #ireland consumer price index (mom) climbed from previous 0.2% to 0.5% in may   #blog #silver #gold #forex
2 - we are so selfish. #orlando #standwithorlando #pulseshooting #orlandoshooting #biggerproblems #selfish #heabreaking   #values #love #
3 - i get to see my daddy today!!   #80days #gettingfed
4 - ouch...junior is angryð#got7 #junior #yugyoem   #omg 
5 - i am thankful for having a paner. #thankful #positive     


In [13]:
import re
#clean text from noise
def clean_text(text):
  text = re.sub(r'[^a-zA-Z\']','',text)
  text = text.lower()
  return text

In [14]:
dataset['clean_text'] = dataset.tweet.apply(lambda x: clean_text(x))

In [15]:
dataset.head(10)

,id,label,tweet,clean_text
0,1,0,@user when a father is dysfunctional and is s...,userwhenafatherisdysfunctionalandissoselfishhe...
1,2,0,@user @user thanks for #lyft credit i can't us...,useruserthanksforlyftcreditican'tusecausetheyd...
2,3,0,bihday your majesty,bihdayyourmajesty
3,4,0,#model i love u take with u all the time in ...,modeliloveutakewithuallthetimeinur
4,5,0,factsguide: society now #motivation,factsguidesocietynowmotivation
5,6,0,[2/2] huge fan fare and big talking before the...,hugefanfareandbigtalkingbeforetheyleavechaosan...
6,7,0,@user camping tomorrow @user @user @user @use...,usercampingtomorrowuseruseruseruseruseruseruse...
7,8,0,the next school year is the year for exams.ð...,thenextschoolyearistheyearforexamscan'tthinkab...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...,wewonlovethelandallincavschampionsclevelandcle...
9,10,0,@user @user welcome here ! i'm it's so #gr...,useruserwelcomeherei'mit'ssogr


In [16]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [17]:
from nltk.corpus import stopwords
len(stopwords.words('english'))

179

In [18]:
stop = stopwords.words('english')

In [19]:
def gen_freq(text):
  word_list = []
  for tw_words in text.split():
    word_list.extend(tw_words)
  word_freq = pd.Series(word_list).value_counts()
  word_freq = word_freq.drop(stop, errors='ignore')
  return word_freq

In [20]:
def any_neg(words):
  for word in words:
    if word in ['n','no','non','not'] or re.search(r"\wn't",word):
      return 1
    else:
      return 0

In [21]:
def is_question(words):
  for word in words:
    if word in ['when','what','how','why','who','where']:
      return 1
    else:
      return 0

In [22]:
def gen_freq(text):
       word_list = []
       for tw_words in text.split():
           word_list.extend(tw_words)
       word_freq = pd.Series(word_list).value_counts()
       word_freq = word_freq.drop(stop, errors='ignore')
       return word_freq

In [25]:
def any_rare(words, rare_100):
  """Checks if any of the words in a list are in the rare_100 list."""
  return any(word in rare_100 for word in words)

word_freq = gen_freq(dataset.clean_text.str)

# 100 most rare words in the dataset
rare_100 = word_freq[-100:]  # last 100 rows/words

# Number of words in a tweet
dataset['word_count']

,word_count
0,1
1,1
2,1
3,1
4,1
...,...
5237,1
5238,1
5239,1
5240,1


In [26]:
word_freq = gen_freq(dataset.clean_text.str)

# 100 most rare words in the dataset
rare_100 = word_freq[-100:]  # last 100 rows/words

# Number of words in a tweet
dataset['word_count'] = dataset.clean_text.str.split().apply(lambda x: len(x))

# Negation present or not
dataset['any_neg'] = dataset.clean_text.str.split().apply(any_neg)

# Prompt present or not
dataset['is_question'] = dataset.clean_text.str.split().apply(is_question)

# Any of the 100 most rare words present or not
dataset['any_rare'] = dataset.clean_text.str.split().apply(lambda x: any_rare(x, rare_100))

# Character count of the tweet
dataset['char_count'] = dataset.clean_text.apply(len)


In [31]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

# Assuming 'label' is your target variable and you want to use
# 'word_count', 'any_neg', 'is_question', 'any_rare', 'char_count' as features
X = dataset[['word_count', 'any_neg', 'is_question', 'any_rare', 'char_count']]
y = dataset['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
model = GaussianNB()
model.fit(X_train, y_train)
pred = model.predict(X_test)

In [32]:
model.predict(X_test[5:10])

array([1, 1, 1, 1, 1])

In [33]:
from sklearn.metrics import accuracy_score

print("Accuracy:", accuracy_score(y_test, pred)*100, "%")

from sklearn.ensemble import RandomForestClassifier
#Create a random forest classifier, 100 trees
clf_rf=RandomForestClassifier()

#Train the model using the training sets
clf_rf.fit(X_train,y_train)

rf_pred=clf_rf.predict(X_test).astype(int)

Accuracy: 45.156369183829135 %


In [34]:
from sklearn.metrics import classification_report, confusion_matrix, \
accuracy_score
print(confusion_matrix(y_test,rf_pred))
print(classification_report(y_test,rf_pred))
print("Accuracy:",accuracy_score(y_test, rf_pred))

from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(class_weight='balanced')
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test) #predicting the values

[[517 224]
 [303 267]]
              precision    recall  f1-score   support

           0       0.63      0.70      0.66       741
           1       0.54      0.47      0.50       570

    accuracy                           0.60      1311
   macro avg       0.59      0.58      0.58      1311
weighted avg       0.59      0.60      0.59      1311

Accuracy: 0.5980167810831426
